In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.applications import VGG16, MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

In [3]:
train_1 = np.load(r'/content/drive/My Drive/Baby Cry/stratified k fold data/train_3.npy')
test_1 = np.load(r'/content/drive/My Drive/Baby Cry/stratified k fold data/test_3.npy')
train_lab_1 = np.load(r'/content/drive/My Drive/Baby Cry/stratified k fold data/train_lab_3.npy')
test_lab_1 = np.load(r'/content/drive/My Drive/Baby Cry/stratified k fold data/test_lab_3.npy')


baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(288,432, 3)))
# add layers for transfer learning
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(2, 2),name = 'averagepool_last')(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(4096, activation="relu")(headModel)
headModel = Dense(512, activation="relu",name = 'dense_512')(headModel)
headModel = Dense(128, activation="relu",name = 'dense_128')(headModel)   # we can experiment with more dense layers if output of average pooling layer is too big
headModel = Dropout(0.3,name = 'dropout_last')(headModel)
headModel = Dense(5, activation="softmax",name = 'output')(headModel)  # we have 5 classes
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
#model.summary()
#------------Optional: We can train only Classification layers with below code or do training from scratch with keras.fit()
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
# for layer in baseModel.layers:
#   layer.trainable = False

###for models with Batch-Norm  
for layer in baseModel.layers:
    if layer.__class__.__name__=='BatchNormalization':
      #print('exist')
      layer.trainable = False
    #else:
      #print('normal layer')  

# adding regularization
regularizer = tf.keras.regularizers.l2(.0001)

for layer in model.layers:
    if layer.trainable == True:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)

#opt=SGD(learning_rate=1e-4, nesterov=True)
#opt=tf.keras.optimizers.Nadam(learning_rate=1e-4)

opt=Adam(lr=1e-4)
model.compile(loss='categorical_crossentropy', optimizer=opt,	metrics=["accuracy"])
model.summary()
import tensorflow as tf
#callback for best Val Accuracy
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
  filepath="/content/best.ckpt",
  save_weights_only=True,
  monitor='val_accuracy',
  mode='max',
  save_best_only=True)
model.fit(train_1, train_lab_1, batch_size=32,	validation_data=(test_1, test_lab_1),	epochs=100, verbose=2, callbacks=[model_checkpoint_callback])
#load the checkpoint from here

#get the latest checkpoint file
checkpoint_dir = os.path.dirname('/content/best.ckpt')
latest = tf.train.latest_checkpoint(checkpoint_dir)

#Create a new model instance
model_latest_checkpoint = model
#model_latest_checkpoint.summary()
# Load the previously saved weights
model_latest_checkpoint.load_weights(latest)
# Re-evaluate the model
loss, acc = model_latest_checkpoint.evaluate(test_1, test_lab_1, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))



Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 288, 432, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 288, 432, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 288, 432, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 144, 216, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 144, 216, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 144, 216, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 72, 108, 128)     

In [4]:
agg_train_1 = np.load(r'/content/drive/My Drive/Baby Cry/68 feat strat k fold data/68_train_3.npy')
agg_test_1 = np.load(r'/content/drive/My Drive/Baby Cry/68 feat strat k fold data/68_test_3.npy')

print(agg_train_1.shape)

(605, 13532)


In [5]:

#freeze layers of vgg16

for layer in model_latest_checkpoint.layers:
  layer.trainable = False


intermediate_output = model_latest_checkpoint.get_layer("flatten").output

In [6]:
#convert array to keras tensor

tensor_train = Input(shape=(agg_train_1.shape[1],))
y = Dense(8192, activation="relu")(tensor_train)
y = Dense(4096,activation = 'relu')(y)
 
#print(tensor_train)

#merge layer output with this array(keras tensor)
merged_output = tf.keras.layers.Concatenate(axis=1)([intermediate_output, y])

x = Dense(512, activation="relu",name = 'MLP_512')(merged_output)
x = Dense(128, activation="relu",name = 'MLP_128')(x)   # we can experiment with more dense layers if output of average pooling layer is too big
x = Dropout(0.3,name = 'dropout_MLP')(x)
out_mlp = Dense(5, activation="softmax",name = 'output')(x)  # we have 5 classes

#build the mlp model
second_model = Model(inputs = tensor_train, outputs=y)

total_model = Model(inputs=[model_latest_checkpoint.input, second_model.input], outputs=out_mlp)
opt=Adam(lr=1e-4)
total_model.compile(loss='categorical_crossentropy', optimizer=opt,	metrics=["accuracy"])
total_model.summary()
#Add 68 feature + MPL model at end 

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 288, 432, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 288, 432, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 288, 432, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 144, 216, 64) 0           block1_conv2[0][0]               
_______________________________________________________________________________________

In [7]:
import tensorflow as tf
#callback for best Val Accuracy
new_model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/total_best.ckpt",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [8]:
print(train_1.shape)
print(agg_train_1.shape)

(605, 288, 432, 3)
(605, 13532)


In [9]:
#total_model
total_model.fit([train_1,agg_train_1], train_lab_1, batch_size=32,	validation_data=([test_1,agg_test_1], test_lab_1),	epochs=20, verbose=2, callbacks=[new_model_checkpoint_callback])

Epoch 1/20
19/19 - 77s - loss: 0.9784 - accuracy: 0.6959 - val_loss: 0.6436 - val_accuracy: 0.8092
Epoch 2/20
19/19 - 33s - loss: 0.6902 - accuracy: 0.8661 - val_loss: 1.2430 - val_accuracy: 0.8224
Epoch 3/20
19/19 - 47s - loss: 1.0106 - accuracy: 0.8975 - val_loss: 1.0101 - val_accuracy: 0.8289
Epoch 4/20
19/19 - 15s - loss: 0.1969 - accuracy: 0.9521 - val_loss: 0.7598 - val_accuracy: 0.8158
Epoch 5/20
19/19 - 15s - loss: 0.3224 - accuracy: 0.9388 - val_loss: 0.8560 - val_accuracy: 0.8289
Epoch 6/20
19/19 - 34s - loss: 0.1173 - accuracy: 0.9587 - val_loss: 0.7433 - val_accuracy: 0.8421
Epoch 7/20
19/19 - 15s - loss: 0.1474 - accuracy: 0.9785 - val_loss: 0.8833 - val_accuracy: 0.8355
Epoch 8/20
19/19 - 15s - loss: 0.0639 - accuracy: 0.9835 - val_loss: 0.6882 - val_accuracy: 0.8421
Epoch 9/20
19/19 - 34s - loss: 0.0393 - accuracy: 0.9884 - val_loss: 0.6632 - val_accuracy: 0.8553
Epoch 10/20
19/19 - 47s - loss: 0.0201 - accuracy: 0.9967 - val_loss: 0.6644 - val_accuracy: 0.8618
Epoch 11/

In [10]:
#load the checkpoint from here

#get the latest checkpoint file
checkpoint_dir = os.path.dirname('/content/total_best.ckpt')
latest = tf.train.latest_checkpoint(checkpoint_dir)

#Create a new model instance
model_latest_checkpoint = total_model
#model_latest_checkpoint.summary()
# Load the previously saved weights
model_latest_checkpoint.load_weights(latest)
# Re-evaluate the model
loss, acc = model_latest_checkpoint.evaluate([test_1,agg_test_1],  test_lab_1, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


5/5 - 2s - loss: 0.5988 - accuracy: 0.8684
Restored model, accuracy: 86.84%
